In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
import tensorflow as tf

In [6]:
import scipy.fftpack

def smooth_data(data, window):
    return np.convolve(y, np.ones(window)/window, mode='same')

# Importing and cleaning data
df = pd.read_csv("short_term_rates.csv")
df1 = df.iloc[:, [0, 4]]
df1 = df1.iloc[5:, :]
df1["Series Description"] = pd.to_datetime(df1["Series Description"])
df1.iloc[:, 1] = pd.to_numeric(df1.iloc[:, 1], errors="coerce")
df1 = df1.dropna()
df1.iloc[:, 1] = df1.iloc[:, 1].astype('float64')

In [7]:
size = 500
padding = 50
total_len = len(df1)
margin = 10

real_x = df1.iloc[:, 0].values
real_y = df1.iloc[:, 1].values

In [8]:
def time_diff(x1, x2):
    diff = x2 - x1
    diff = diff / np.timedelta64(1, 'D')
    return diff

train_x = []
train_y = []

a_list = []
ab_list = []
sigma_list = []

for i in range(padding, total_len - padding - margin):
    start = i - padding
    end = i + size + padding

    x = real_x[start: end]
    y = real_y[start:end]
    smooth_y = smooth_data(y,padding)
    smooth_y = smooth_y[padding:-1 * padding]
    x = x[padding:-1 * padding]
    y = y[padding:-1 * padding]
    
    total_days_diff = time_diff(x[0], x[len(x) - 1])
    dt = 1 / total_days_diff
    deltaT = (np.diff(x) / np.timedelta64(1, 'D')) * dt
    deltaR = np.diff(smooth_y)
    R = smooth_y[: -1]
    Rdt = R * deltaT
    
    A = np.column_stack([deltaT, Rdt])
    AT = A.T

    AT_A = AT @ A
    AT_A = AT_A.astype(float)
    inverse_AT_A = np.linalg.inv(AT_A)
    p = (inverse_AT_A @ AT)
    parameters = p @ deltaR
    
    a = -parameters[1]  
    ab = parameters[0]
    a_list.append(a)
    ab_list.append(ab)
    
    diffy = y - smooth_y
    abssum = np.sum(np.abs(diffy)) / size
    sigma = abssum / np.sqrt(dt)
    sigma_list.append(sigma)
    
    train_x.append(real_y[i:i+size])
    train_y.append([a, ab, sigma])

    

In [9]:
# Model normalization

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

train_x = np.array(train_x)
train_y = np.array(train_y)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# actua 
from tensorflow.keras import layers, models
import numpy as np

model = models.Sequential([
    layers.Input(shape=(500,)),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(3)  # Final layer with 3 outputs
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

history = model.fit(train_X, train_y, epochs=20, batch_size=32, validation_split=0.2)
